In [5]:
import pandas as pd
import numpy as np
from pathlib import Path

ROOT_FOLDER = "NYPL-menus"
# ROOT_FOLDER = "NYPL-menus-cleaned"

MENU = 1
MENU_PAGE = 3
MENU_ITEM = 2
DISH = 0

# Data Profiling Workflow Instructions

## Accessing Datasets

Each dataset can be accessed from the dataset collection using the following keys:

```python
dataset["MENU"]          # Menu data
dataset["MENU_PAGE"]     # Menu page data
dataset["MENU_ITEM"]     # Menu item data 
dataset["DISH"]          # Dish data
```

## Profiling the Cleaned Dataset

Update the ROOT_FOLDER variable to point to your cleaned data directory:

```python
ROOT_FOLDER = "NYPL-menus-cleaned"  # Path to cleaned data
```

In [6]:
dataset = []

for filename in sorted(Path(ROOT_FOLDER).iterdir()):
    print(filename)
    if filename.name.endswith(".csv"):
        dataset.append(pd.read_csv(filename, na_values=[""]))


NYPL-menus\Dish.csv
NYPL-menus\Menu.csv
NYPL-menus\MenuItem.csv
NYPL-menus\MenuPage.csv


In [3]:
# IC 1: Temporal consistency in Dish
ic1_violations = dataset[DISH][dataset[DISH]["first_appeared"] > dataset[DISH]["last_appeared"]]

print(f"Violations found: {len(ic1_violations)}")
ic1_violations.head(10)

Violations found: 0


,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price


In [4]:
dataset[MENU_ITEM][dataset[MENU_ITEM]["dish_id"].isin(dataset[DISH].loc[ic1_violations.index, "id"])]

,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


In [5]:
# IC 2: Date Outliner in Menu

# Extract first 4 digits of call_number (if not null)
dataset[MENU]["call_prefix"] = dataset[MENU]["call_number"].str[:4]

# Extract first 4 digits of date (if not null)
dataset[MENU]["date_prefix"] = dataset[MENU]["date"].str[:4]

# Check constraint: call_prefix == date_prefix when both exist
ic2_violations = dataset[MENU][
    dataset[MENU]["call_number"].notna() & 
    dataset[MENU]["date"].notna() & 
    dataset[MENU]["call_prefix"].str.isnumeric() &
    dataset[MENU]["date"].str.match(r"[^1]+") & 
    (dataset[MENU]["call_prefix"] != dataset[MENU]["date_prefix"])
]

print(f"Violations found: {len(ic2_violations)}")
ic2_violations[["id", "call_number", "date", "call_prefix", "date_prefix"]].head()

Violations found: 0


,id,call_number,date,call_prefix,date_prefix


In [6]:
# IC 3: Date Blank with call number year in Menu

# Extract first 4 digits of call_number (if not null)
dataset[MENU]["call_prefix"] = dataset[MENU]["call_number"].str[:4]

# Extract first 4 digits of date (if not null)
dataset[MENU]["date_prefix"] = dataset[MENU]["date"].str[:4]

# Check constraint: date should not be blank when call_prefix is numeric
ic3_violations = dataset[MENU][
    dataset[MENU]["call_number"].notna() & 
    dataset[MENU]["date"].isna() &
    dataset[MENU]["call_prefix"].str.isnumeric()
]

print(f"Violations found: {len(ic3_violations)}")
ic3_violations[["id", "call_number", "date", "call_prefix", "date_prefix"]].head(20)

Violations found: 0


,id,call_number,date,call_prefix,date_prefix


In [7]:
# IC 4: Date Blank with no date info in call number in Menu

# Extract first 4 digits of call_number (if not null)
dataset[MENU]["call_prefix"] = dataset[MENU]["call_number"].str[:4]

# Extract first 4 digits of date (if not null)
dataset[MENU]["date_prefix"] = dataset[MENU]["date"].str[:4]

# Check constraint: date should not be blank when call_prefix is numeric
ic4_violations = dataset[MENU][
    dataset[MENU]["call_number"].notna() & 
    dataset[MENU]["date"].isna() & (
    (dataset[MENU]["call_prefix"] == 'Zand') |
    (dataset[MENU]["call_prefix"] == 'Soet') |
    (dataset[MENU]["call_prefix"] == 'soet') |
    (dataset[MENU]["call_prefix"] == 'Bara') |
    (dataset[MENU]["call_prefix"] == '_wot'))
]

print(f"Violations found: {len(ic4_violations)}")
dataset[MENU].loc[ic4_violations.index]

Violations found: 0


,id,name,sponsor,event,venue,place,physical_description,occasion,notes,call_number,...,date,location,location_type,currency,currency_symbol,status,page_count,dish_count,call_prefix,date_prefix


In [8]:
# IC 5: Date range outsite of 1890-1970 in Menu
ic5_violations = dataset[MENU][
    dataset[MENU]["date"].notna() & 
    (dataset[MENU]["date"].str[:4].astype(int) < 1890) | 
    (dataset[MENU]["date"].str[:4].astype(int) > 1970)
]

print(f"Violations found: {len(ic5_violations)}")
dataset[MENU].loc[ic5_violations.index]

Violations found: 1248


,id,name,sponsor,event,venue,place,physical_description,occasion,notes,call_number,...,date,location,location_type,currency,currency_symbol,status,page_count,dish_count,call_prefix,date_prefix
39,12503,NaN,POLICE DEPARTMENT OF THE CITY OF NEW YORK,SEVENTH ANNUAL DINNER,GOVT;,DELMONICO'S,FOL; 4.75 x 7.25;,NaN,SEAL ON COVER; FRENCH; INCLUDES WINES SERVED W...,1888-0010,...,1888-01-23,Police Department Of The City Of New York,NaN,NaN,NaN,complete,3,30,1888,1888
48,12515,NaN,THE ALBANY,LUNCH,?,"DENVER, COLO;",CARD;3.5 X 5;,NaN,NaN,1888-0611,...,1888-10-15,The Albany,NaN,NaN,NaN,complete,2,30,1888,1888
49,12516,NaN,REVERE HOUSE,COMPLIMENTARY BANQUET GIVEN BY THE CITY GOVERN...,RESTAURANT,"BOSTON, MA",BROADSIDE; ILLUS; 4.25 X 11.75,NaN,MENU PRINTED IN BLACK ON CREAM SILK RIBBON WIT...,1865-0001,...,1865-09-28,Parker House,NaN,Dollars,$,complete,4,422,1865,1865
147,12635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1888-10-15,The Albany,NaN,NaN,NaN,complete,2,30,NaN,1888
148,12636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1865-06-09,Revere House,NaN,Dollars,$,complete,4,403,NaN,1865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11875,29396,NaN,Hotel Mas De Vence,NaN,NaN,NaN,19x14cm folded; 19x28cm open,NaN,NaN,1995-0016_wotm,...,1995-07-03,Hotel Mas De Vence,NaN,NaN,NaN,complete,3,4,1995,1995
11877,29398,NaN,Outpost,NaN,NaN,NaN,21.5x12cm folded; 21.5x35.5cm open,NaN,NaN,1995-0006_wotm,...,1995-04-22,Outpost,NaN,Dollars,$,complete,3,59,1995,1995
11878,29399,NaN,Que Pasta's,NaN,NaN,NaN,21.5x18cm folded; 21.5x35.5cm open,NaN,NaN,1988-0001_wotm,...,1988-01-01,Que Pasta's,NaN,Dollars,$,complete,3,64,1988,1988
11880,29401,NaN,Le Riveria,NaN,NaN,NaN,21x15cm folded; 21x30cm open,NaN,NaN,1990-0012_wotm,...,1990-06-06,Le Riveria,NaN,NaN,NaN,complete,3,7,1990,1990


In [16]:
# IC 6: Date range outsite of 1890-1970 in Dish
ic6_violations = dataset[DISH][
    (dataset[DISH]["first_appeared"] == 0) | 
    (dataset[DISH]["last_appeared"] == 0)
]

ic6_violations_2 = dataset[MENU_ITEM][
    dataset[MENU_ITEM]["dish_id"].isin(dataset[DISH].loc[ic6_violations.index, "id"])
]

ic6_violations_3 = dataset[MENU_PAGE][
    dataset[MENU_PAGE]["id"].isin(ic6_violations_2["menu_page_id"])
]

ic6_violations_4 = dataset[MENU][
    dataset[MENU]["id"].isin(ic6_violations_3["menu_id"])
]

print(f"Violations found: {len(ic6_violations_4)}")
# dataset[DISH].loc[ic6_violations.index]
# dataset[MENU_ITEM].loc[ic6_violations_2.index]
# dataset[MENU_PAGE].loc[ic6_violations_3.index]
dataset[MENU].loc[ic6_violations_4.index]

Violations found: 1828


,id,name,sponsor,event,venue,place,physical_description,occasion,notes,call_number,keywords,language,date,location,location_type,currency,currency_symbol,status,page_count,dish_count
127,12610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1900-04-20,Hotel Imperial,NaN,NaN,NaN,complete,2,15
299,12841,NaN,HOTEL MARLBOROUGH,BREAKFAST,COMMERCIAL,"[66 STREET AND BROADWAY,NEW YORK,NY]",CARD;ILL;COL;7X11;,DAILY;,COAT OF ARMS;DATE HANDWRITTEN BY FB;,1900-2226,NaN,NaN,1900-03-02,Hotel Marlborough,NaN,Dollars,$,complete,2,317
336,12882,NaN,CLAREMONT HOTEL,MENU,COMMERCIAL,NaN,CARD; ILLUS; 6X11;,NaN,PRICED MENU & PRICED WINE LISTS; PRICES APPEAR...,1900-2740,NaN,NaN,1900-04-09,Claremont Hotel,NaN,Dollars,$,complete,2,226
343,12896,NaN,HOTEL MARIE ANTOINETTE,LUNCHEON,COMMERCIAL,66TH STREET AND BR6ADWAY,CARD; 6.5 X 9.75;,NaN,PRICED MENU; CREST;,1900-2015,NaN,NaN,1900-02-18,Hotel Marie Antoinette,NaN,Dollars,$,under review,2,82
350,12905,NaN,SAN REMO HOTEL,DINNER,COMMERCIAL,"75TH ST & CENTRAL PARK WEST, NY",FOLDER; 6.5 X 10.5;,DAILY,NaN,1900-1042,NaN,NaN,1900-02-13,San Remo Hotel,NaN,Dollars,$,complete,4,243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17237,35097,Hotel Earlington,Hotel Earlington,NaN,NaN,NaN,NaN,NaN,New York Nautical School,1913-0325_wotm,NaN,NaN,1913-03-29,Hotel Earlington,NaN,NaN,NaN,complete,8,19
17238,35099,Waldorf Astoria,Waldorf Astoria,NaN,NaN,NaN,NaN,NaN,Missouri Society of the City of New York,1913-0327_wotm,NaN,NaN,1913-03-29,Waldorf Astoria,NaN,NaN,NaN,complete,8,19
17239,35102,Palace Hotel,Palace Hotel,NaN,NaN,NaN,NaN,NaN,Northern California Hotel Association,1913-0330_wotm,NaN,NaN,1913-03-29,Palace Hotel,NaN,NaN,NaN,complete,3,21
17240,35103,Waldorf Astoria,Waldorf Astoria,NaN,NaN,NaN,NaN,NaN,Wall Paper Manufacturers Association,1913-0331_wotm,NaN,NaN,1913-03-29,Waldorf Astoria,NaN,NaN,NaN,complete,11,22
